In [3]:
using Pkg
Pkg.activate("../")

using Revise
using HiddenMarkovModel

 Activating environment at `/mnt/matylda3/ikocour/HiddenMarkovModel/Project.toml`


In [5]:
C,D,N = 3,2,42
gmm = GMM(C, randn(D,N))
X = randn(D,N)
lhpf(gmm1, X) |> display
avlh(gmm1, X)

42-element Array{Float64,1}:
 0.09906677589385958
 0.0036816605453356514
 0.11800344615594381
 0.0008933610150143949
 0.0032415224290777015
 0.02438627638123945
 0.003895429135815981
 0.08271032717419538
 0.04027582252654482
 0.0663420372112898
 0.10906418860007605
 0.015965918134160848
 0.026827356248816273
 ⋮
 0.11003460537310819
 0.0683032662601018
 0.12537219423225215
 0.12744738957372923
 0.04943016781127091
 0.015128781586413318
 0.11150766709953555
 0.06953504779538787
 0.07308493151504224
 0.007452937814454119
 0.09786954819733287
 0.004566207250821361

2.1895172900157367e-71

In [7]:
using Distributions

In [8]:
?Distribution

search: Distribution Distributions MatrixDistribution NonMatrixDistribution



```
Distribution{F<:VariateForm,S<:ValueSupport} <: Sampleable{F,S}
```

`Distribution` is a `Sampleable` generating random values from a probability distribution. Distributions define a Probability Distribution Function (PDF) to implement with `pdf` and a Cumulated Distribution Function (CDF) to implement with `cdf`.


In [ ]:
function train!(pdfs::Vector{Distribution}, hmm::HiddenMarkovModel, Xs::Vector{AbstractArray})
    stats = [GmmStats(pdf) for pdf in pdfs]
    tll_acc = 0
    
    for X in Xs
        # E-step
        llh = zeros(nstates(hmm), size(X,2))
        for state in hmm
            llh[state.id, :] = logpdf(state.d, X)
        end
        
        γ, tll, logα, logβ = forward_backward(hmm.Tr, llh;
            fs=nstates(hmm), π=[1.0; zeros(nstates(hmm)-1)])
        
        tll_acc += tll
        
        # accumulate statistics
        for state in hmm
            stats[state.id] += compute_stats(state.d, γ[state.id, :], X)
        end
    end
    
    # M-step
    for state in hmm
        hmm.G[state.id] = fit(state.d, stats[state.id])
    end
    # TODO: update Tr
    return tll_acc/size(Xs,1)
end

In [9]:
?αβrecursion

"αβrecursion" can be typed by \alpha<tab>\beta<tab>recursion

search: αβrecursion



```
αβrecursion(graph, llh[, pruning = ...])
```

Baum-Welch algorithm in  the log domain.
